### Notes
* Part 1 is to update the 'parcels_geography.csv' file with the new zoningmods and nodev attributes
* Part 2 is to generate geospacial files to use in GIS. The output file dissolves parcels on 'pba50zoningmodcat', and contains "parcel count" and "total acres" for each dissolved geometry. The script enables two approaches for dissolving: 
    * 1) Load the 'UrbanSim_input_Zoning\outputs\parcel_zoningmods.shp' into ArcGIS and dissolve.
    * 2) Dissolve in geopands and then export. 

In [2]:
import pandas as pd
import geopandas as gpd
import os
import fiona

In [55]:
juris_raw = pd.read_csv('inputs/jurisId.csv')
pg_old = pd.read_csv('inputs/07_11_2019_parcels_geography.csv')
pba50_att_raw = pd.read_csv('inputs/p10_pba50_attr_20200407.csv')
p10_geo = gpd.read_file('inputs/p10_geo_shp.shp')

### Update 'parcels_geography.csv'

In [83]:
pba50_att = pba50_att_raw[['geom_id_s','pda_id', 'tpp_id', 'exp_id', 'exp_score', 'opp_id', 'zoningmodcat', 
                           'perffoot', 'perfarea', 'mapshell', 'tpa_id', 'perfarea2', 'alt_zoning', 'zonetype', 'pubopp_id', 
                           'puboppuse', 'juris_id', 'hra_id', 'trich_id', 'cat_id', 'chcat', 'zoninghzcat', 
                           'gg_id', 'tra_id', 'sesit_id', 'ppa_id', 'exp2020_id', 'pba50chcat', 'exsfd_id', 'chcatwsfd', 
                           'pba50zoningmodcat', 'nodev']]
pg_temp = pg_old[['Unnamed: 0','geom_id','urbanized']]
juris = juris_raw[['jurisdiction_id','juris_id']]
p10_geo.PARCEL_ID = p10_geo.PARCEL_ID.apply(lambda x: int(x))
p10_geo.geom_id_s = p10_geo.geom_id_s.apply(lambda x: int(x))

In [84]:
pg = pg_temp.merge(pba50_att, left_on = 'geom_id', right_on = 'geom_id_s', how = 'left').merge(
    juris, on = 'juris_id', how = 'left').merge(p10_geo, on = 'geom_id_s', how = 'left')

In [85]:
pg_csv = pg[['geom_id','PARCEL_ID','juris_id','gg_id','tra_id','sesit_id','ppa_id','exp2020_id',
            'exsfd_id','pba50chcat', 'pba50zoningmodcat','nodev','jurisdiction_id']]
pg_csv.to_csv('outputs/2020_04_10_parcels_geography.csv')

### Generate 'parcel_zoningmods.shp' for mapping

In [86]:
# select required fields
pg_geo = gpd.GeoDataFrame(pg[['geom_id','PARCEL_ID','juris_id','gg_id','tra_id','sesit_id','ppa_id','exp2020_id','exsfd_id',
                              'pba50chcat', 'pba50zoningmodcat','nodev','jurisdiction_id','ACRES','geometry']], geometry='geometry')
pg_geo[['ACRES']] = pg_geo[['ACRES']].fillna(value=0)
pg_geo = pg_geo.where(pd.notnull(pg_geo), None)
pg_geo.to_file('outputs/parcel_zoningmods.shp')

In [104]:
# create a 
for_join = pg[['pba50zoningmodcat','pba50chcat']].drop_duplicates()
print(for_join.shape)
for_join.to_csv(r'C:\Users\ywang\Documents\Files_for_Py\UrbanSim_input_Zoning\outputs\zoningmods_for_join.csv')

stats = pg_geo[['PARCEL_ID','ACRES','pba50zoningmodcat']].groupby(['pba50zoningmodcat']).agg({'PARCEL_ID':'count', 'ACRES': 'sum'}).reset_index()
stats = stats.merge(for_join, on = 'pba50zoningmodcat', how = 'left')
stats.columns = ['pba50zoningmodcat', 'parcel_count','acres','pba50chcat']
stats_nonZero = stats.query('parcel_count > 0 & acres > 0')
print(stats_nonZero.shape)

In [106]:
# check missing pba50zoningmod
print(pg.loc[pg.pba50zoningmodcat.isnull()].shape[0] == 0)

True


In [5]:
# create mapping file
pg_geo['modTemp'] = pg_geo[['gg_id', 'tra_id', 'sesit_id', 'ppa_id']].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)

gdf = gpd.GeoDataFrame(pg_geo[['PARCEL_ID','modTemp','geometry']], geometry='geometry')
print(gdf.shape)
gdf2 = gdf.loc[gdf.geometry.notnull(),:]
print(gdf2.shape)

In [ ]:
# dissolve by pba50zoningmodca
gdf2['geometry'] = gdf2.buffer(0.01)
try:
    zoningmod_dis = gdf2.dissolve(by='modTemp',as_index=False)
except:
    print('error: ', gdf2.modTemp)

display(zoningmod_dis.head())

C:\Users\ywang\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
df = pd.DataFrame(zoningmod_dis, copy=True)
df_split = pd.concat([df['modTemp'].str.split('_', expand=True), df['geometry'],df['PARCEL_ID']], axis=1)
display(df_split.head())

In [211]:
df_split.columns = ['juris_id', 'gg_id', 'tra_id', 'sesit_id', 'ppa_id', 'exp2020_id', 'exsfd_id', 'chcatwsfd','NA','geometry','PARCEL_ID']
df_split = df_split.merge(juris_name[['jurisdiction','juris']], left_on = 'juris_id', right_on = 'juris',how = 'left')
df_split.drop(columns = ['juris','juris_id'], inplace = True)

,geom_id,jurisdiction_id_old,pda_id,tpp_id,exp_id,opp_id,zoningmodcat,perffoot,perfarea,urbanized,...,juris_id,gg_id,tra_id,sesit_id,ppa_id,exp2020_id,exsfd_id,pba50zoningmodcat,nodev,jurisdiction_id
0,10305106092872,41992,NaN,NaN,NaN,NaN,41992NANANANA,1,0,1,...,livr,NA,NA,HRADR,NA,in,NA,livrNANAHRADRNAinNA,0,41992
1,11107351665227,41992,NaN,NaN,NaN,NaN,41992NANANANA,1,0,1,...,livr,NA,NA,DR,NA,in,NA,livrNANADRNAinNA,0,41992
2,11030175960628,33000,NaN,NaN,NaN,NaN,33000NANANANA,1,0,0,...,hayw,NA,NA,NA,NA,in,NA,haywNANANANAinNA,0,33000
3,6381677629073,97,NaN,NaN,NaN,NaN,00097NANANANA,0,0,0,...,uson,NA,NA,DR,NA,out,NA,usonNANADRNAoutNA,0,97
4,314875459798,26000,NaN,b1,NaN,NaN,26000NAb1NANA,1,1,1,...,frem,NA,NA,HRADR,NA,in,NA,fremNANAHRADRNAinNA,1,26000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1956203,17158666132196,2252,NaN,NaN,NaN,NaN,02252NANANANA,1,0,1,...,anti,NA,NA,DR,NA,in,sfd,antiNANADRNAinsfd,0,2252
1956204,16389503450045,16000,NaN,NaN,NaN,NaN,16000NANANANA,1,0,1,...,conc,NA,NA,DR,NA,in,sfd,concNANADRNAinsfd,0,16000
1956205,1496694834659,52582,NaN,NaN,NaN,NaN,52582NANANANA,1,0,1,...,nova,NA,NA,HRADR,NA,in,NA,novaNANAHRADRNAinNA,0,52582
1956206,10694584892329,13,NaN,NaN,NaN,NaN,00013NANANANA,1,0,1,...,ucnc,NA,NA,HRADR,NA,inun,sfd,ucncNANAHRADRNAinunsfd,0,13


In [ ]:
dissolved = gpd.GeoDataFrame(df_split, geometry='geometry')
dissolved.to_file('outputs/pba50_zoningmods_diss.shp')